In [1]:
from bs4 import BeautifulSoup
import requests
from db import PythonProblems
from datetime import datetime
from urllib.parse import urljoin
from tqdm import tqdm

### Creating and connecting to database

In [32]:
db = PythonProblems('python.sqlite', 'init_tables.sql')

### Getting the pages to be crawled

In [33]:
%%time
urls = [
       # "https://www.w3resource.com/python-exercises/python-basic-exercises.php", 
       # "https://www.w3resource.com/python-exercises/basic/", # commit d66ecae4b3ee871ca6d94d90ac9fde7d28dd1c67
       # "https://www.w3resource.com/python-exercises/string/",
       # "https://www.w3resource.com/python-exercises/list/", # commit 42e8eb9a78d7474cecf2c373e3c9c9b4652423e7
       # "https://www.w3resource.com/python-exercises/dictionary/", # commit 6490b5779a9f472edbe7cd286d579557b2b53411
       # "https://www.w3resource.com/python-exercises/tuple/", # commit aa04882c57893437f896b46e755433a64b34c1c6
       # "https://www.w3resource.com/python-exercises/sets/", # commit ebd13e285ccaa3437b6b7076d01b18d5f9c06277
       # "https://www.w3resource.com/python-exercises/array/", # commit 5d43e87ca2614bc42b11a3ada50616c48eac5dd8
        "https://www.w3resource.com/python-exercises/python-conditional-statements-and-loop-exercises.php"]
pms = []
for idx, url in enumerate(urls):
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html5lib')
    solution_links, problem_comparison, problems = get_problems(soup)
    
    #if idx == 0:
    #    # Bug-fix, problem is written wrong in the first page
    #    problems[145]["content"] = "Write a Python program to test if a variable is a list or tuple or a set."
    
    #if idx == 5:
    #if idx == 0:
    #    # Bug-fix, problem is written wrong in the first page
    #    problems[21]["content"] = "Write a Python program to remove an empty tuple(s) from a list of tuples."
    
    # Dictionary has problem repetition, but different solutions
    if idx == 4:
        solutions, potential_mismatch = get_solutions(solution_links, problem_comparison, problems)
    else:
        solutions, potential_mismatch = get_solutions(solution_links, problem_comparison)
        
    pms.append(potential_mismatch)
    insert(problems, solutions)
#db.close_connection()

100%|██████████| 45/45 [00:00<00:00, 5668.15it/s]
43it [00:41,  1.03it/s]

Got 43 problems inserted, 43 solutions inserted, 0 problems repeated and 0 solutions repeated
CPU times: user 3.8 s, sys: 32 ms, total: 3.83 s
Wall time: 42.5 s


## Functions

### Problems and solutions links

In [3]:
def get_problems(soup):
    exercises = soup.find_all("strong")
    problem_list = []
    solution_links = []
    problem_comparison = []

    title = soup.find_all("h2", class_="heading")[0].text.split("[")[0].strip()

    for e in tqdm(exercises):
        # Find solutions links
        solution_link = e.findNext("a")
        try:
            while solution_link.text.strip() != "Click me to see the sample solution":
                solution_link = solution_link.findNext("a")
        except AttributeError:
            continue
        solution_links.append(solution_link)

        # Get problem
        problem_text = ""
        problem_text = e.parent.text
        if e.parent.next_sibling and e.parent.next_sibling.name == "pre":
            problem_text += e.parent.next_sibling.text

        # Get original text for comparison in solution's page
        editor_partition = problem_text.partition("Go to the editor")
        #Go to the editor is written
        if editor_partition[2] != '':
            problem_comparison.append(editor_partition[0].split('.')[1].strip())
        # Go to the editor is not written
        else:
            problem_comparison.append(problem_text.split('.')[1])

        problem_text = problem_text.replace("Go to the editor", "")
        problem_text = problem_text.replace("Click me to see the sample solution", "")
        problem_text = problem_text.partition('.')[2].strip()
        problem_list.append(problem_text)

    problems = []

    for problem_text in problem_list:
        problems.append({"title": title, "link": url,
                         "content": problem_text, "retrieved_date": datetime.now(),
                         "crawler": "W3Resource"})
        
    return solution_links, problem_comparison, problems

### Solutions

In [8]:
def get_solutions(solution_links, problem_comparison, problems=None):
    solutions = []
    potential_mismatch = []
    
    if problems:
        problems_list = [p['content'] for p in problems]
    
    for idx, link in tqdm(enumerate(solution_links)):
        link_sol = urljoin(url, link['href'])
        response = requests.get(link_sol)
        soup = BeautifulSoup(response.text, 'html5lib')

        # Compare problem inside with original problem
        compare_problem = soup.find_all("h2")[0].findNext().text
        if compare_problem[:-1].strip() != problem_comparison[idx].strip():
            potential_mismatch.append(link_sol + '\n' + compare_problem[:-1].strip() + '\n' + problem_comparison[idx] + '\n\n')

        code = soup.find_all("code")
        
        if problems:
            idx_sol = problems_list.index(problems_list[idx])
        else:
            idx_sol = idx
        
        solutions.append({"content": code[0].text, "link": link_sol,
                          "retrieved_date": datetime.now(), "ignore": False,
                          "idx": idx_sol})
    return solutions, potential_mismatch

### Insert rows

In [5]:
def insert(problems, solutions):
    db.insert_rows(problems, solutions)
    tp, ts, rp, rs = db.populate()
    print("Got %d problems inserted, %d solutions inserted, %d problems repeated and %d solutions repeated" % (
          tp, ts, rp, rs))

### Control check

In [30]:
for item in pms[0]:
    print(item)

https://www.w3resource.com/python-exercises/array/python-array-exercise-1.php
Write a Python program to create an array of 5 integers and display the array items. Access individual element through indexes
Write a Python program to create an array of 5 integers and display the array items


https://www.w3resource.com/python-exercises/array/python-array-exercise-5.php
Write a Python program to get the current memory address and the length in elements of the buffer used to hold an array’s contents and also find the size of the memory buffer in bytes
Write a Python program to get the current memory address and the length in elements of the buffer used to hold an array?s contents and also find the size of the memory buffer in bytes




In [31]:
db.rows

[{'problem': {'content': 'Write a Python program to create an array of 5 integers and display the array items. Access individual element through indexes.',
   'crawler': 'W3Resource',
   'link': 'https://www.w3resource.com/python-exercises/array/',
   'retrieved_date': datetime.datetime(2017, 12, 11, 17, 7, 46, 380519),
   'title': 'Python Array'},
  'solution': {'content': 'from array import *\narray_num = array(\'i\', [1,3,5,7,9])\nfor i in array_num:\n    print(i)\nprint("Access first three items individually")\nprint(array_num[0])\nprint(array_num[1])\nprint(array_num[2])\n',
   'ignore': False,
   'link': 'https://www.w3resource.com/python-exercises/array/python-array-exercise-1.php',
   'problem_id': 565,
   'retrieved_date': datetime.datetime(2017, 12, 11, 17, 7, 47, 511228)}},
 {'problem': {'content': 'Write a Python program to append a new item to the end of the array.',
   'crawler': 'W3Resource',
   'link': 'https://www.w3resource.com/python-exercises/array/',
   'retrieved_

In [26]:
solution_links

[<a href="python-sets-exercise-1.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-2.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-3.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-4.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-5.php" target="_blank"> Click me to see the sample solution</a>,
 <a href="python-sets-exercise-6.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-7.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-8.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-9.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-10.php" target="_blank">Click me to see the sample solution</a>,
 <a href="python-sets-exercise-11.php" target="_